<a href="https://colab.research.google.com/github/fuadwa/Amazon/blob/main/bankchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

# API endpoint for getting latest rates
url = 'https://api.frankfurter.app/latest'

import requests

# API endpoint for getting latest rates
url = 'https://api.frankfurter.app/latest'

# Base currency and target currencies
base_currency = 'USD'
target_currencies = ['EUR', 'GBP', 'JPY']

# Request parameters
params = {
    'base': base_currency,
    'symbols': ','.join(target_currencies)
}

# Function to retrieve and format currency rates
def get_currency_rates():
    # Send GET request to the API
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        
        # Get the rates from the response
        rates = data['rates']
        
        # Format the rates
        formatted_rates = []
        for currency, rate in rates.items():
            formatted_rates.append(f"{currency}: {rate}")
        
        return formatted_rates
    else:
        return []


In [ ]:
!pip install spacy



In [ ]:
!python -m spacy download en_core_web_lg



In [ ]:
import en_core_web_lg # Large SpaCy model for English language
import numpy as np
import re # regular expressions
import spacy # NLU library

from collections import defaultdict
from sklearn.svm import SVC # Support Vector Classification model

In [ ]:
output_format = "You: {input}\nVirtual Assistant: {output}\n" + "_"*50

In [65]:
responses_exact = {
    "selam": "selam endem koyachehugh",
    "how can i activate my user id?": "If you have opened an account recently, you will find your Internet Banking User ID/ Password in the Welcome Kit provided to you at the time of Account opening.",
    "default": "I did not understand. Please try again.",
    "give me daily currency": lambda: "\n".join(get_currency_rates()),
}

def respond_exact(text):
    response = responses_exact.get(text.lower(), responses_exact['default'])
    
    # Check if the response is a function (currency rates)
    if callable(response):
        return response()
    
    return response

In [66]:
print(respond_exact("selam"))
print("_"*50)
print(respond_exact("How can I activate my user ID?"))
print("_"*50)
print(respond_exact("I forgot my password."))
print("_"*50)
print(respond_exact("give me daily currency"))

selam endem koyachehugh
__________________________________________________
If you have opened an account recently, you will find your Internet Banking User ID/ Password in the Welcome Kit provided to you at the time of Account opening.
__________________________________________________
I did not understand. Please try again.
__________________________________________________
EUR: 0.93015
GBP: 0.80749
JPY: 139.75


In [ ]:
intent_keywords = {
    'internet_banking': ['net', 'internet', 'login', 'password', 'activate'],
    'timings': ['open hours', 'timings', 'schedule', 'closing time','open','time','close','timing','when']
}
# Create a dictionary of patterns
patterns = {intent: re.compile('|'.join(keys)) for intent, keys in intent_keywords.items()}

# Define a function to find the intent of a message
def get_intent_re(message):
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if pattern.search(message):
            return(intent)
    else:
        return('default')

responses_re = {
    "internet_banking":"You can contact the customer care at 9898999898 for queries regarding internet banking.",
    "timings": "The bank opens at 10:00 hours and closes at 18:00 hours",
    "default":"I did not understand. Please try again."
}

def respond_re(text):
    response = responses_re.get(get_intent_re(text))
    return(output_format.format(input=text, output=response))

In [ ]:
print(respond_re("How do I activate my net banking account?"))
print(respond_re("When will the office be open?"))
print(respond_re("How do I cancel my membership?"))

You: How do I activate my net banking account?
Virtual Assistant: You can contact the customer care at 9898999898 for queries regarding internet banking.
__________________________________________________
You: When will the office be open?
Virtual Assistant: The bank opens at 10:00 hours and closes at 18:00 hours
__________________________________________________
You: How do I cancel my membership?
Virtual Assistant: I did not understand. Please try again.
__________________________________________________


In [ ]:
# Create training data
training_sentences = [
    "How do I activate my account?",
    "I do not have a user ID and password. How do I get one?",
    "I have got my Internet Banking User ID and Password; tell me what do I do?",
    "How do I register a new biller in Internet Banking?",
    "The Bank's Internet Banking User ID has expired.", 
    "When does the office open?",
    "What are the office's open hours?",
    "When does the bank close?",
    "What are the timings of the office?",
    "What is the schedule that the office follows?"
    
]
training_intents = [
    "internet_banking",
    "internet_banking",
    "internet_banking",
    "internet_banking",
    "internet_banking",
    "timings",
    "timings",
    "timings",
    "timings",
    "timings"   
]

In [ ]:
nlp = en_core_web_lg.load()

In [ ]:
# Initialize the array with zeros: X
X_train = np.zeros((len(training_sentences), 
              nlp('sentences').vocab.vectors_length))

for i, sentence in enumerate(training_sentences):
    # Pass each each sentence to the nlp object to create a document
    doc = nlp(sentence)
    # Save the document's .vector attribute to the corresponding row in X
    X_train[i, :] = doc.vector

In [ ]:
# Create a support vector classifier
clf = SVC(C=1, gamma="auto", probability=True)

# Fit the classifier using the training data
clf.fit(X_train, training_intents)

SVC(C=1, gamma='auto', probability=True)

In [ ]:
def get_intent_ml(text):
    doc = nlp(text)
    return(clf.predict([doc.vector])[0])

In [ ]:
responses_ml = {
    "internet_banking":"Go to your nearest branch of Ethiopian Commercial Bank. Make sure to bring your identification documents such as your ID card, passport, or any other required documents.",
    "timings": "The bank opens at 10:00 hours and closes at 18:00 hours",
    "default":"I did not understand. Please try again."
}

def respond_ml(text):
    response = responses_ml.get(get_intent_ml(text), responses_ml["default"])
    return(output_format.format(input=text, output=response))

In [ ]:
print(respond_ml("How do I activate my net banking account?"))
print(respond_ml("When will the office be open?"))
print(respond_ml("How do I cancel my membership?"))

You: How do I activate my net banking account?
Virtual Assistant: You can contact the customer care at 9898999898 for queries regarding internet banking.
__________________________________________________
You: When will the office be open?
Virtual Assistant: The bank opens at 10:00 hours and closes at 18:00 hours
__________________________________________________
You: How do I cancel my membership?
Virtual Assistant: You can contact the customer care at 9898999898 for queries regarding internet banking.
__________________________________________________


In [ ]:
def get_intent_ml_2(text):
    """
        Returns the intent from a given text, unless the model is not sure, in which case 'default' is returned
    """
    doc = nlp(text)
    max_proba = max(clf.predict_proba([doc.vector])[0])
    if(max_proba == 0.5):
        return('default')
    else:
        return(clf.predict([doc.vector])[0])

def respond_ml_2(text):
    response = responses_ml.get(get_intent_ml_2(text), responses_ml["default"])
    return(output_format.format(input=text, output=response))

In [ ]:
print(respond_ml(  'credit'))
print(respond_ml_2('credit'))

You: credit
Virtual Assistant: You can contact the customer care at 9898999898 for queries regarding internet banking.
__________________________________________________
You: credit
Virtual Assistant: You can contact the customer care at 9898999898 for queries regarding internet banking.
__________________________________________________


In [ ]:
def get_all_entities(text):
    """
        Get all entities in a given text, in a text: label_ dictionary
    """
    doc = nlp(text)
    
    d = defaultdict(list)
    for ent in doc.ents:
        d[ent.label_].append(ent.text)
    return(d)

In [ ]:
test_ents = get_all_entities('when will the portal close tonight?, is the bank open on saturday and sunday?')
print(sorted(test_ents.items()))

[('DATE', ['saturday', 'sunday']), ('TIME', ['tonight'])]


In [ ]:
policy = {
    ("internet_banking", "time and date"): "You can contact the customer care at 9898999898 for queries regarding internet banking.",
    ("internet_banking", "time only"): "You can contact the customer care at 9898999898 for queries regarding internet banking.",
    ("internet_banking", "date only"): "You can contact the customer care at 9898999898 for queries regarding internet banking.",
    ("internet_banking", "none"): "What?",
    ("timings", "time and date"): "The bank opens at 10:00 hours and closes at 18:00 hours.(Monday to Friday)",
    ("timings", "time only"): "The bank opens at 10:00 hours and closes at 18:00 hours.(Monday to Friday)",
    ("timings", "date only"): "The bank opens at 10:00 hours and closes at 18:00 hours.(Monday to Friday)",
    ("timings", "none"): "When?",
    ("timings", "none"): "What do you want?",
}

In [ ]:
def respond_ml_3(text):
    """Check for specification of date and time
        If not specified, ask for clarification
    """
    intent = get_intent_ml_2(text)
        
    if intent != 'default':
        entities = get_all_entities(text)
        if 'TIME' in entities and 'DATE' in entities:
            specification = 'time and date'
            time = ' and '.join(entities['DATE']) + ' at ' + ' and '.join(entities['TIME'])
        elif 'TIME' in entities:
            specification = 'time only'
            time = ' and '.join(entities['TIME'])
        elif 'DATE' in entities:
            specification = 'date only'
            time = ' and '.join(entities['DATE'])
        else:
            specification = 'none'
            time = ""
    else:
        specification = 'none'
        time = ""
    
    response = policy.get((intent, specification)) + ' ' + time
    return(output_format.format(input=text, output=response))

In [68]:
preferences = {"monday": "credit card applications"}


In [72]:
print(respond_ml_3("What are the timings for internet banking?"))

You: What are the timings for internet banking?
Virtual Assistant: What do you want? 
__________________________________________________
